<a href="https://www.kaggle.com/code/vedanshmannem/gse243217?scriptVersionId=217949766" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

Quick Summary of script for the future:
1. Run the General for identifying all candidate miRNAs
2. Run "Orderings" for specific families i.e. different diseases or severe vs. mild disease

Should produce csv of all the miRNA values (rsquared, pval, etc.)

In [ ]:
fixgenenames=function(rns){
rns<-toupper(rns)
x<-grep("-SEP|-MAR|-DEC",rns)
g.names<-rns[x]
g.names.1<-gsub("MAR","MARCH",gsub("SEP","SEPT",sapply(strsplit(g.names,"-"),function(x)x[2])))
g.names.2<-sapply(strsplit(g.names,"-"),function(x)x[1])
g.names<-paste(g.names.1,g.names.2,sep="")
rns[x]=g.names
return(rns)
}

frame_gene_mat=function(genes,platform=platform){
if(platform=="RNA_Seq"){
gene.mat=gset[which(plat_genes%in%genes),]}
if(platform=="microarray"){
gene.mat=exprs(gset)[which(plat_genes%in%genes),]}

# if(!class(gene.mat)%in%c("matrix","data.frame")){

# if(class(gene.mat)!="matrix"|class(gene.mat)!="data.frame"){
# gene.mat<-t(data.frame(gene.mat))
# rownames(gene.mat)<-genes}

gene.mat.names=plat_genes[which(plat_genes%in%genes)]
gene.mat.1={}
for (i in genes){
if (length(which(gene.mat.names==i))>1){
gene.mat.1=rbind(gene.mat.1,colMeans(gene.mat[which(gene.mat.names==i),],na.rm=T))}else{
gene.mat.1=rbind(gene.mat.1,gene.mat[which(gene.mat.names==i),])}
}
rownames(gene.mat.1)=genes
return(gene.mat.1)
}

zscore_func = function(gene_mat) {
score_mat=t(apply(gene_mat,1,scale))
return(score_mat)}

In [ ]:
library("limma");library("preprocessCore");
library("stringr");library("scales");
library("tibble");library("matrixStats")

In [ ]:
phen = read.csv("/kaggle/input/gse243219/GSE243217_phen.csv")
gset = read.csv("/kaggle/input/normalizedgse243219/GSE243217_counts_data.csv")
ncphen = read.csv("/kaggle/input/gse243219/GSE243218_phen.csv")
ncgset = read.csv("/kaggle/input/normalizedgse243219/GSE243218_counts_data.csv")
ncgset = ncgset[which(rowSums(ncgset)!=0),]
nrow(ncgset) # Should be 1850
ncplat_genes = read.csv('/kaggle/input/gse243218platgenes/GSE243218_platgenes.csv')
ncplat_genes = ncplat_genes[,-1]
scoremat = read.csv("/kaggle/input/243217-salutoscoremat/243217-fullsaluto-scoremat.csv")

scoremat = scoremat[,-1]

Code to identify candidate biomarkers

In [ ]:
# Saluto gene family
cnames = c("SAS.1",   "MAS.1",   "Age_IL6up",   "SenMayo",   "Inflammageup",   "ELup",   "Age_IL6down",   "t.TCHlow",   "Inflammagedown",   "IMM.AGE",   "t.TCHhigh",   "ELdown",   "ISG_33",   "ISG_23",   "GL_Mortality",   "GL_Survival",   "TLR_Mortality",   "TLR_Survival",   "Muthu_aHR_Mort",   "Muthu_aHR_Surv",   "Prot_chromatin.remodel",   "Prot_DNA.repair",   "Prot_stem.cell.pop.maint",   "Prot_TCR.signaling",   "Prot_Tcell.costim",   "Prot_Tcell.activ",   "Det_inflam.resp",   "Det_pos.reg.inflam.resp",   "Det_signal.transduct",   "Det_pos.reg.NFkB",   "Det_def.resp.bact",   "Det_angiogenesis",   "IHM_up",   "IHM_dn",   "jPC1_up",   "jPC1_dn",   "PM2_up",   "PM2_dn",   "LD78b_up",   "LD78b_dn",   "MIP1a_up",   "MIP1a_dn",   "PARC_up",   "PARC_dn",   "CD5L_up",   "CD5L_dn",   "IL18Ra_up",   "IL18Ra_dn",   "SAA_up",   "SAA_dn",   "CRP_up",   "CRP_dn",   "Sialoadhesin_up",   "Sialoadhesin_dn",   "RDW_up",   "RDW_dn",   "NK.ct_up",   "NK.ct_dn",   "NK.pct_up",   "NK.pct_dn",   "SRS.CAP",   "SRS.FP",   "SRS.Extended",   "SRS1.Up",   "SRS2.Up") # Salutoscores

## Run for general
rsquared <- data.frame(matrix(NA, nrow = 1850, ncol = 65))
intercept <- data.frame(matrix(NA, nrow = 1850, ncol = 65))
slope <- data.frame(matrix(NA, nrow = 1850, ncol = 65))
pval <- data.frame(matrix(NA, nrow = 1850, ncol = 65))

colnames(rsquared) = cnames
colnames(intercept) = cnames
colnames(slope) = cnames
colnames(pval) = cnames

## Run for orderings
# rsquared = c()
# intercept <- c()
# slope <- c()
# pval <- c()

In [ ]:
# score vs nc
for(i in 1:65) {
    for(j in 1:1850) {
        yval = scoremat[i,]
        xval = ncgset[j,]
        fit=lm(as.numeric(yval)~1+as.numeric(xval))
        rsquared[j,i] = summary(fit)$adj.r.squared
        intercept[j,i] = summary(fit)$coefficients[1,1]
        slope[j,i] = summary(fit)$coefficients[2,1]
        pval[j,i] = summary(fit)$coefficients[2,4]
    }
}

# Run for orderings
# for(j in 1:1850) {
#     xval = ncgset[j,which(phen$disease!="healthy donor")]
#     zval = phen$disease[which(phen$disease!="healthy donor")]
#     fit=lm(as.numeric(xval)~1+factor(zval))
#     rsquared = c(rsquared, summary(fit)$adj.r.squared)
#     intercept = c( intercept, summary(fit)$coefficients[1,1])
#     slope = c(slope, summary(fit)$coefficients[2,1])
#     pval = c(pval,summary(fit)$coefficients[2,4])
# }

In [ ]:
qvals <- data.frame(matrix(NA, nrow = 1850, ncol = 65))
# Run for general
for(i in 1:ncol(pval)) {
    qvals[,i] = p.adjust(pval[,i], method="BH")
}
colnames(qvals) = colnames(pval)

# Run for orderings
# qvals = c()
# qvals = p.adjust(pval, method="BH")
# qvals


In [ ]:
# Run for general
newdata = {}
cols = c("ncRNA")
for(i in 1:65) {
    newdata = cbind(newdata, slope[,i], rsquared[,i], pval[,i], qvals[,i])
    cols = c(cols, paste(c("slope", "rsquared", "pval", "qvals"), by=colnames(rsquared)[i]))
}
newdata = cbind(ncplat_genes, newdata)
colnames(newdata) = cols
newdata

# # Run for ordering
# newdata = data.frame(matrix(NA, nrow = 1850, ncol = 6))
# newdata[,1] = ncplat_genes
# newdata[,2] = rsquared
# newdata[,3] = intercept
# newdata[,4] = slope
# newdata[,5] = pval
# newdata[,6] = qvals
# colnames(newdata) = c("miRNA", "rsquared", "intercept", "slope", "pval", "qvals")
# newdata

In [ ]:
write.csv(newdata, "/kaggle/working/243217-saulto-general.csv")

Generalized Additive Models

In [ ]:
library(mgcv)
library(ggplot2)

In [ ]:
pval <- data.frame(matrix(NA, nrow = 1850, ncol = 1))
cnames = c()
for(j in 1:1850) {
    fulldata1 = data.frame(
        SAS = t(scoremat[1,]),
        miRNA = t(ncgset[j,])
    )
    colnames(fulldata1) = c("SAS", "miRNA")
    if(length(unique(fulldata1$miRNA)) >= 10) {
        mod_gam1 = gam(fulldata1$SAS ~ s(fulldata1$miRNA, bs = "cr"), data=fulldata1)
        cnames = c(cnames, ncplat_genes[j])
        pval[j,1] = k.check(mod_gam1)[, 'p-value']
    } else {
        pval[j,1] = NA
    }
}

rownames(pval) = (ncplat_genes)

In [ ]:
summary(mod_gam1)
names(mod_gam1)
# mod_gam1$edf

In [ ]:
cols = which(pval<0.05)
for(i in cols) {
    fulldata1 = data.frame(
        SAS = t(scoremat[1,]),
        miRNA = t(ncgset[i,])
    )
    colnames(fulldata1) = c("SAS", "miRNA")
    
    p = ggplot(data = fulldata1, mapping = aes(x = SAS, y = miRNA)) +
      geom_point(size = 0.5, alpha = 0.5) +
      geom_smooth(method = "gam", formula = y ~ s(x, bs = "cr"))
    print(p)
}

In [ ]:
# colnames(pval) = c("p-val")
write.csv(pval, "/kaggle/working/gampval.csv")

Logistic Regression Code

In [ ]:
install.packages("tidyverse")   
install.packages("caTools") 
library(tidyverse)  
library(caTools)

In [ ]:
# Formatting data
disease_state = as.character(phen$disease)
disease_state = gsub("healthy donor", "Healthy", disease_state)
disease_state = gsub("COVID-19", "Diseased", disease_state)
disease_state = gsub("Sepsis", "Diseased", disease_state)

data = data.frame(
    mir150 = t(ncgset[236,]),
    mir1275 = t(ncgset[132,]),
    disease = disease_state
)
colnames(data) = c("mir150","mir1275","disease")
data$disease <- ifelse(data$disease == "Diseased", 1, 0) # Diseased = 1 & Healthy = 0

# Splitting data
set.seed(123) 
split = sample.split(data$disease, SplitRatio = 0.8) 
train_data = subset(data, split == TRUE)
test_data = subset(data, split == FALSE)
dim(train_data)
dim(test_data)

# Fitting model
logistic_model <- glm(disease ~ mir150 + mir1275, 
                      data = train_data, 
                      family = binomial)
summary(logistic_model)

# Testing model
pred_prob <- predict(logistic_model, newdata = test_data, type = "response")
pred_class <- ifelse(pred_prob > 0.5, 1, 0)
test_data$pred = pred_class
conf_matrix <- table(Predicted = test_data$pred, Actual = test_data$disease)
print(conf_matrix)

accuracy <- sum(diag(conf_matrix)) / sum(conf_matrix)
cat("Accuracy:", accuracy, "\n")

WGCNA

In [ ]:
BiocManager::install("WGCNA")
BiocManager::install("magrittr")
library(magrittr)  
BiocManager::install("dplyr")
library(dplyr)  
BiocManager::install("tidyr")
library(tidyr)  
install.packages("ggplot2")
library(ggplot2)

In [ ]:
mRNA = t(gset)
miRNA = t(ncgset)

In [ ]:
powers = c(c(1:10), seq(from = 12, to = 20, by = 2))

sft = pickSoftThreshold(
  miRNA,
  #blockSize = 30,
  powerVector = powers,
  verbose = 5
  )

par(mfrow = c(1,2));
cex1 = 0.9;

plot(sft$fitIndices[, 1],
     -sign(sft$fitIndices[, 3]) * sft$fitIndices[, 2],
     xlab = "Soft Threshold (power)",
     ylab = "Scale Free Topology Model Fit, signed R^2",
     main = paste("Scale independence")
)
text(sft$fitIndices[, 1],
     -sign(sft$fitIndices[, 3]) * sft$fitIndices[, 2],
     labels = powers, cex = cex1, col = "red"
)
abline(h = 0.90, col = "red")
plot(sft$fitIndices[, 1],
     sft$fitIndices[, 5],
     xlab = "Soft Threshold (power)",
     ylab = "Mean Connectivity",
     type = "n",
     main = paste("Mean connectivity")
)
text(sft$fitIndices[, 1],
     sft$fitIndices[, 5],
     labels = powers,
     cex = cex1, col = "red")

In [ ]:
picked_power = 5
temp_cor <- cor       
cor <- WGCNA::cor        
netwk <- blockwiseModules(miRNA,           
                          power = picked_power,            
                          networkType = "signed",

                          # == Tree and Block Options ==
                          deepSplit = 2,
                          pamRespectsDendro = F,
                          # detectCutHeight = 0.75,
                          minModuleSize = 30,
                          maxBlockSize = 4000,

                          # == Module Adjustments ==
                          reassignThreshold = 0,
                          mergeCutHeight = 0.25,

                          # == TOM == Archive the run results in TOM file (saves time)
                          saveTOMs = T,
                          saveTOMFileBase = "ER",

                          # == Output Options
                          numericLabels = T,
                          verbose = 3)

cor <- temp_cor     # Return cor function to original namespace

In [ ]:
# Convert labels to colors for plotting
mergedColors = labels2colors(netwk$colors)
# Plot the dendrogram and the module colors underneath
plotDendroAndColors(
  netwk$dendrograms[[1]],
  mergedColors[netwk$blockGenes[[1]]],
  "Module colors",
  dendroLabels = FALSE,
  hang = 0.03,
  addGuide = TRUE,
  guideHang = 0.05 )

In [ ]:
module_df <- data.frame(
  gene_id = names(netwk$colors),
  colors = labels2colors(netwk$colors)
)

module_df[1:5,]
#>            gene_id    colors
#> 1 AC149818.2_FG001      blue
#> 2 AC149829.2_FG003      blue
#> 3 AC182617.3_FG001      blue
#> 4 AC186512.3_FG001 turquoise
#> 5 AC186512.3_FG007 turquoise


In [ ]:
# Get Module Eigengenes per cluster
MEs0 <- moduleEigengenes(miRNA, mergedColors)$eigengenes

# Reorder modules so similar modules are next to each other
MEs0 <- orderMEs(MEs0)
module_order = names(MEs0) %>% gsub("ME","", .)

# Add treatment names
MEs0$treatment = row.names(MEs0)

# tidy & plot data
mME = MEs0 %>%
  pivot_longer(-treatment) %>%
  mutate(
    name = gsub("ME", "", name),
    name = factor(name, levels = module_order)
  )

mME %>% ggplot(., aes(x=treatment, y=name, fill=value)) +
  geom_tile() +
  theme_bw() +
  scale_fill_gradient2(
    low = "blue",
    high = "red",
    mid = "white",
    midpoint = 0,
    limit = c(-1,1)) +
  theme(axis.text.x = element_text(angle=90)) +
  labs(title = "Module-trait Relationships", y = "Modules", fill="corr")

In [ ]:
which(submod$gene_id %in% ncplat_genes)

In [ ]:
modules_of_interest = c("turquoise", "pink", "purple")

# Pull out list of genes in that module
submod = module_df %>%
  subset(colors %in% modules_of_interest)

row.names(module_df) = module_df$gene_id

# Get normalized expression for those genes
subexpr = ncgset[which(submod$gene_id %in% ncplat_genes),]

submod_df = data.frame(subexpr) %>%
  mutate(
    gene_id = row.names(.)
  ) %>%
  pivot_longer(-gene_id) %>%
  mutate(
    module = module_df[gene_id,]$colors
  )

In [ ]:
submod_df %>% ggplot(., aes(x=name, y=value, group=gene_id)) +
  geom_line(aes(color = module),
            alpha = 0.2) +
  theme_bw() +
  theme(
    axis.text.x = element_text(angle = 90)
  ) +
  facet_grid(rows = vars(module)) +
  labs(x = "sample",
       y = "normalized expression")